In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import utils
import plotting
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import mplhep as hep
import vector

In [14]:
#Load your dataset
samples = {
    
    "qcd": [
        "QCD_HT-1000to1200",
        "QCD_HT-1200to1500",
        "QCD_HT-1500to2000",
        "QCD_HT-2000",
        "QCD_HT-200to400",
        "QCD_HT-400to600",
        "QCD_HT-600to800",
        "QCD_HT-800to1000",
    ],
    "ttbar": [
        "TTto4Q",
        "TTtoLNu2Q",
        
    ],  
    "hh4b": [
        "GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_Private",
    ],
}

year = "2022"#
dir_name = "24Jan18_v12"
path_to_dir = f"/eos/uscms/store/user/cmantill/bbbb/skimmer/{dir_name}/"
dirs = {path_to_dir: samples}

filters = [
    [
        ("('ak8FatJetPt', '0')", ">=", 300),
        ("('ak8FatJetPt', '1')", ">=", 300),
        ("('ak8FatJetMsd', '0')", ">=", 50),
        ("('ak8FatJetPNetMass', '1')", ">=", 50),
        ("('ak8FatJetPNetXbb', '0')", ">=", 0.8),  
        
        #wasn't getting any hh4b events because of the filter
    ]
]

# columns to load                                                                                                                                                                                                                                     
# the parquet files are too big so we can only load a few columns at a time without consumming much memory
load_columns = [
    ("weight", 1),
	("ak8FatJetMsd", 2),
    ("ak8FatJetPNetMass", 2),
	("ak8FatJetPNetXbb", 2),                                                                                                                                                                                                                     
]
# reformat into ("column name", "idx") format for reading multiindex columns                                                                                                                                                                          
columns = []
for key, num_columns in load_columns:
    for i in range(num_columns):
        columns.append(f"('{key}', '{i}')")


events_dict = {}
for input_dir, samples in dirs.items():
   
    events_dict = {
        **events_dict,
        # this function will load files (only the columns selected), apply filters and compute a weight per event
        **utils.load_samples(input_dir, samples, year, filters=filters),
    }
print("Keys:")
print(events_dict.keys())
samples_loaded = list(events_dict.keys())
#print(samples_loaded)
keys_loaded = list(events_dict[samples_loaded[0]].keys())

#print(events_dict['qcd'])
#This section prints all keys for inspection
#print(f"Keys in events_dict")
#for i in keys_loaded:
#    print(i)


# Assuming df has features and a binary target column named 'target'
#features = df.drop('target', axis=1)
#target = df['target']

['GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_Private', 'QCD_HT-1000to1200', 'QCD_HT-1200to1500', 'QCD_HT-1500to2000', 'QCD_HT-2000', 'QCD_HT-200to400', 'QCD_HT-400to600', 'QCD_HT-600to800', 'QCD_HT-800to1000', 'TTto4Q', 'TTtoLNu2Q', 'VBFHHto4B_CV_1_C2V_1_C3_1_TuneCP5_13p6TeV_madgraph-pythia8', 'WminusH_Hto2B_Wto2Q_M-125', 'WplusH_Hto2B_Wto2Q_M-125', 'ZH_Hto2B_Zto2Q_M-125', 'ggZH_Hto2B_Zto2Q_M-125']
Loading QCD_HT-1000to1200
/eos/uscms/store/user/cmantill/bbbb/skimmer/24Jan18_v12//2022/QCD_HT-1000to1200/parquet
Filter anbd Sample
<class 'filter'>
None
QCD_HT-1000to1200
Loading QCD_HT-1200to1500
/eos/uscms/store/user/cmantill/bbbb/skimmer/24Jan18_v12//2022/QCD_HT-1200to1500/parquet
Filter anbd Sample
<class 'filter'>
None
QCD_HT-1200to1500
Loading QCD_HT-1500to2000
/eos/uscms/store/user/cmantill/bbbb/skimmer/24Jan18_v12//2022/QCD_HT-1500to2000/parquet
Filter anbd Sample
<class 'filter'>
None


QCD_HT-1500to2000
Loading QCD_HT-200to400
/eos/uscms/store/user/cmantill/bbbb/skimmer/24Jan18_v12//2022/QCD_HT-200to400/parquet
Filter anbd Sample
<class 'filter'>
None
QCD_HT-200to400
Loading QCD_HT-400to600
/eos/uscms/store/user/cmantill/bbbb/skimmer/24Jan18_v12//2022/QCD_HT-400to600/parquet
Filter anbd Sample
<class 'filter'>
None
QCD_HT-400to600
Loading QCD_HT-600to800
/eos/uscms/store/user/cmantill/bbbb/skimmer/24Jan18_v12//2022/QCD_HT-600to800/parquet
Filter anbd Sample
<class 'filter'>
None


QCD_HT-600to800
Loading TTto4Q
/eos/uscms/store/user/cmantill/bbbb/skimmer/24Jan18_v12//2022/TTto4Q/parquet
Filter anbd Sample
<class 'filter'>
None
TTto4Q
Loading TTtoLNu2Q
/eos/uscms/store/user/cmantill/bbbb/skimmer/24Jan18_v12//2022/TTtoLNu2Q/parquet
Filter anbd Sample
<class 'filter'>
None
TTtoLNu2Q
Loading GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_Private
/eos/uscms/store/user/cmantill/bbbb/skimmer/24Jan18_v12//2022/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_Private/parquet
Filter anbd Sample
<class 'filter'>
None
GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_Private
Keys:
dict_keys(['qcd', 'ttbar'])


In [15]:
def make_vector(events: pd.DataFrame, obj: str):
    """Create a ``vector`` object from the columns of the dataframe"""
    mstring = "PNetMass" if obj == "ak8FatJet" else "Mass"

    return vector.array(
        {
            "pt": events[f"{obj}Pt"],
            "phi": events[f"{obj}Phi"],
            "eta": events[f"{obj}Eta"],
            "M": events[f"{obj}{mstring}"],
        }
    )

In [16]:
#DEFINE FEATURES & TARGET


events = pd.concat([events_dict["hh4b"], events_dict["qcd"], events_dict["ttbar"]], keys=['hh4b','qcd', 'ttbar'])

#events['target'] = 0  # Default to 0 (background)
#events.loc['hh4b', 'target'] = 1  # Set to 1 for 'hh4b' samples (signal)

#print(events)
# Extracting all unique keys
all_keys = set(key for row in events['data_column'] if isinstance(row, dict) for key in row.keys())

# Display all unique keys
print(all_keys)

# fatjets sorted by xbb
fatjets = make_vector(events, "ak8FatJet")

# H1 candidate
h1 = fatjets[:, 0]
# H2 candidate
h2 = fatjets[:, 1]

h1_xbb = events.ak8FatJetPNetXbb[0]
h2_xbb = events.ak8FatJetPNetXbb[1]

# dihiggs candidate object
hh = h1 + h2

#define the dataframe
df = pd.DataFrame()

# define feature variables
df['HHlogPt'] = np.log(hh.pt)
df['HHeta'] = hh.eta
df['HHmass'] = hh.mass
#missing transverse energy
#jet 1 Tau_32
#jet 2 Tau_32
# jet 1 softdrop mass
df['H1logPt'] = np.log(h1.pt)
df['H1eta'] = h1.eta
df['H1Xbb'] = h1_xbb
# Jet1 QCD b score
# Jet 1 QCDbb score
# Jet 1 QCDothers score
df['H2logPt'] = np.log(h2.pt)
df['H1Pt_HHmass'] = h1.pt/hh.mass
df['H2Pt_HHmass'] = h2.pt/hh.mass
df['H1Pt/H2Pt'] = h1.pt/h2.pt


df.replace(np.inf, 10000, inplace=True)
df.replace(-np.inf, 10000, inplace=True)



KeyError: 'hh4b'

In [ ]:
#LOAD FEATURES INTO PD DF

features = df

#define target
target = events['target']

In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
# Initialize the BDT model
bdt_model = XGBClassifier(
    n_estimators= 196,
    max_depth= 17,
    learning_rate=0.1,
    subsample=0.5,
    random_state=42
)



In [ ]:
# Train the model
bdt_model.fit(X_train, y_train)

In [ ]:
# Make predictions
predictions = bdt_model.predict(X_test)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, predictions)

In [ ]:
print(predictions)

In [ ]:
probabilities = bdt_model.predict_proba(X_test)

print(probabilities[:,0])

In [ ]:
y_scores = bdt_model.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

#histogram for y scores signal & background 
#rank by importance

#number of b-jets that can be identified in resolved
#another feather cos(theta_star)

#Plottting
plt.figure()
plt.plot(tpr, fpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlabel('Signal (HH)')
plt.ylabel('Background (QCD&ttbar)')
plt.title('ROC')
plt.legend(loc="upper left")
plt.yscale('log')
plt.savefig('plots/ROC_BDT.png')
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV
import numpy as np

# Load data
iris = load_iris()
X, y = iris.data, iris.target

# Define the model
model = RandomForestClassifier()

# Define the hyperparameters and their ranges
search_space = {
    'n_estimators': (10, 200),  # Integer range
    'max_depth': (5, 50)        # Integer range
}

# Set up Bayesian optimization
bayes_search = BayesSearchCV(model, search_space, n_iter=32, scoring='accuracy', cv=5, n_jobs=-1, random_state=0)

# Perform the Bayesian optimization
bayes_search.fit(X, y)

# Best parameters
print("Best parameters:", bayes_search.best_params_)


In [ ]:
#determine importance of the features

importances = bdt_model.feature_importances_

feature_names = df.columns
feature_importance = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)

for feature in feature_importance:
    print(f"{feature[0]}: {feature[1]:.3f}")


In [ ]:
hh4b_scores = bdt_model.predict_proba(X_test.loc['hh4b'])[:, 1]
qcd_scores = bdt_model.predict_proba(X_test.loc['qcd'])[:, 1]
ttbar_scores = bdt_model.predict_proba(X_test.loc['ttbar'])[:, 1]

In [ ]:
plt.figure()
plt.hist(hh4b_scores, bins=40, histtype='step', linewidth=1.5, color='darkblue')
plt.hist(qcd_scores, bins=40, histtype='step', linewidth=1.5, color='red')
plt.hist(ttbar_scores, bins=40, histtype='step', linewidth=1.5, color='darkgreen')
plt.legend(['hh4b', 'qcd', 'ttbar'])
plt.ylabel('Frequency')
plt.xlabel('Prediciton Score')
plt.title('Model Predictions Histogram')
plt.yscale('log')
plt.show()
